In [1]:
ls

Untitled.ipynb                sample_submission.csv*        test.csv.zip
product_descriptions.csv*     sample_submission.csv.zip     train.csv*
product_descriptions.csv.zip  submission.csv                train.csv.zip
relevance_instructions.docx   test.csv*


In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
import snowballstemmer

In [3]:
df_train = pd.read_csv('train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('test.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv('product_descriptions.csv')

In [4]:
stemmer = snowballstemmer.EnglishStemmer()

In [5]:
def str_stemmer(s):
	return " ".join([stemmer.stemWord(wrd) for wrd in s.lower().split()])

In [6]:
def str_common_word(str1, str2):
	return sum(int(str2.find(word)>=0) for word in str1.split())

In [7]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

In [8]:
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')

In [9]:
df_all.head()

,id,product_title,product_uid,relevance,search_term,product_description
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,"Not only do angles make joints stronger, they ..."
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,"Not only do angles make joints stronger, they ..."
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,BEHR Premium Textured DECKOVER is an innovativ...
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Update your bathroom with the Delta Vero Singl...
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,Update your bathroom with the Delta Vero Singl...


In [14]:
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stemmer(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stemmer(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stemmer(x))

df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)

df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title']+"\t"+df_all['product_description']

df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))

df_all = df_all.drop(['search_term','product_title','product_description','product_info'],axis=1)



In [15]:
df_all.head()

,id,product_uid,relevance,len_of_query,word_in_title,word_in_description
0,2,100001,3.00,2,1,1
1,3,100001,2.50,2,1,1
2,9,100002,3.00,2,1,1
3,16,100005,2.33,3,1,1
4,17,100005,2.67,3,3,2


In [16]:
num_train = df_train.shape[0]

In [19]:
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']

y_train = df_train['relevance'].values
X_train = df_train.drop(['id','relevance'],axis=1).values
X_test = df_test.drop(['id','relevance'],axis=1).values

In [77]:
df_test.drop('relevance',axis=1,inplace=True)
df_test.drop('id',axis=1,inplace=True)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [78]:
df_test

,product_uid,len_of_query,word_in_title,word_in_description
74067,100001,3,0,1
74068,100001,3,1,1
74069,100001,3,1,1
74070,100001,3,2,2
74071,100001,4,2,2
74072,100001,2,0,1
74073,100003,4,4,2
74074,100003,3,3,1
74075,100003,4,1,0
74076,100004,2,2,2


In [22]:
import xgboost as xgb

In [ ]:
xdtest=xgb.DMatrix(X_test)
xdtrain=xgb.DMatrix(X_train,label=y_train)

In [ ]:
df_test.drop

In [23]:
params = {}
params["objective"] ="reg:linear"
params["min_child_weight"] = 120
params["subsample"] = 1
params["colsample_bytree"] = .75
params["silent"] = 0
params["max_depth"] = 4
params["eta"]=0.15
params["gamma"]=1
plst=list(params.items())

In [ ]:
bst=xgb.train(plst,xdtrain,num_boost_round=400)
test_preds=bst.predict(xdtest)

In [ ]:
tbst=xgb.train(plst,xdtrain,num_boost_round=400)
preds=tbst.predict(xdtest)
out=[]
for i in preds:
    closest= min(kk, key=lambda x:abs(x-i))
    out.append(closest)

In [ ]:
test_preds=bst.predict(xdtest)

In [ ]:
test_preds

In [ ]:
test_preds.std(),y_train.std()

In [91]:
pd.DataFrame({"id": id_test, "relevance": resps1}).to_csv('submission.csv',index=False)

In [92]:
pd.read_csv('submission.csv')

,id,relevance
0,1,1.33
1,4,2.00
2,5,2.00
3,6,2.50
4,7,2.00
5,8,2.00
6,10,3.00
7,11,3.00
8,12,1.67
9,13,3.00


In [ ]:
testdf=df_train.copy()

In [38]:
y_trainft.max()

12

In [39]:
kimba2=[i-1 for i in kimba]

In [45]:
le.inverse_transform(kimba2).max()

3.0

In [24]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [25]:
y_trainft=le.fit_transform(y_train)

In [81]:
max(kimba)

13

In [46]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_train,kimba,test_size=0.33,random_state=0)

In [47]:
xd_test=xgb.DMatrix(x_test,y_test)
xd_train=xgb.DMatrix(x_train,label=y_train)

In [83]:
xd_test=xgb.DMatrix(df_test)
xd_train=xgb.DMatrix(df_train,label=kimba)

In [67]:
params = {}
params["objective"] ="reg:linear"
params["min_child_weight"] =240
params["subsample"] = 1
params["colsample_bytree"] = 0.75
params["silent"] = 0
params["max_depth"] = 4
params["eta"]=0.1
params["gamma"]=0
plst=list(params.items())

In [68]:
tbst=xgb.train(plst,xd_train,num_boost_round=400)
preds=tbst.predict(xd_test)
out=[]
for i in preds:
    closest= min(kk, key=lambda x:abs(x-i))
    out.append(closest)
print (mean_squared_error(out, y_test)**0.5)
np.array(out).std(),y_test.std()

2.88647025104


(1.4100893463955864, 3.1860030413147515)

In [69]:
pd.Series(out).unique()

array([ 8,  9, 10,  7,  6, 11,  5])

In [91]:
mean_squared_error(resps1, paka)**0.5

0.54543634455059531

In [66]:
paka=le.inverse_transform(y_test)

ValueError: y contains new labels: [13]

In [ ]:
rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0)
clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=25)
clf.fit(x_train, y_train)
preds = clf.predict(x_test)
out=[]
for i in preds:
    closest= min(kk, key=lambda x:abs(x-i))
    out.append(closest)
print (mean_squared_error(out, y_test)**0.5)
np.array(out).std(),y_test.std()

In [ ]:
kk

In [53]:
kk=pd.Series(y_train).unique()

In [ ]:
out=[]
for i in ll:
    closest= min(kk, key=lambda x:abs(x-i))
    out.append(closest)

In [ ]:
out

In [13]:
from sklearn.metrics import mean_squared_error

In [ ]:
print (mean_squared_error(out, y_test)**0.5)
np.array(out).std(),y_test.std()

In [ ]:
print (mean_squared_error(preds, y_test)**0.5)
preds.std(),y_test.std()

In [52]:
offsets = np.array([0.5]*13)


In [53]:
len(offsets)

13

In [ ]:
pd.Series(y_train).describe()

In [ ]:
paka[16]

In [85]:
tbst=xgb.train(plst,xd_train,num_boost_round=400)
train_preds=tbst.predict(xd_train)#,ntree_limit=tbst.best_iteration)
test_preds=tbst.predict(xd_test)#,ntree_limit=tbst.best_iteration)
test_preds = np.clip(test_preds, -0.99, 13.99)
train_preds = np.clip(train_preds, -0.99, 13.99)

data1 = np.vstack((train_preds, train_preds,kimba))
for j in range(13):
        data1[1, data1[0].astype(int)==j] = data1[0, data1[0].astype(int)==j] + offsets[j]
for j in range(13):
        train_offset = lambda x: -apply_offset(data1, x, j)
        offsets[j] = fmin(train_offset, offsets[j])

ll=data1[0].round().astype(int)
k=pd.Series([0]*(len(df_test)))

data = np.vstack((test_preds, test_preds, k))
for j in range(13):
        data[1, data[0].astype(int)==j] = data[0, data[0].astype(int)==j] + offsets[j]

resps = np.round(np.clip(data[1], 1, 13))


Optimization terminated successfully.
         Current function value: -0.441702
         Iterations: 9
         Function evaluations: 26
Optimization terminated successfully.
         Current function value: -0.441702
         Iterations: 9
         Function evaluations: 26
Optimization terminated successfully.
         Current function value: -0.441702
         Iterations: 9
         Function evaluations: 26
Optimization terminated successfully.
         Current function value: -0.441702
         Iterations: 9
         Function evaluations: 26
Optimization terminated successfully.
         Current function value: -0.441702
         Iterations: 9
         Function evaluations: 26
Optimization terminated successfully.
         Current function value: -0.441731
         Iterations: 12
         Function evaluations: 33
Optimization terminated successfully.
         Current function value: -0.443349
         Iterations: 17
         Function evaluations: 41
Optimization terminated successf

In [86]:
resps1=le.inverse_transform([i-1 for i in resps.astype(int)])

In [68]:
paka=le.inverse_transform([i-1 for i in y_test])

In [90]:
resps1.std()

0.49106338221933687

In [74]:
mean_squared_error(resps1, paka)**0.5

0.56691646701354925

In [72]:
paka.std()

0.53129424357750676

In [12]:
from scipy.optimize import fmin


In [11]:
        def eval_wrapper(yhat, y):
                y = np.array(y)
                y = y.astype(int)
                yhat = np.array(yhat)
                yhat = np.clip(np.round(yhat), np.min(y), np.max(y)).astype(int)
                return quadratic_weighted_kappa(yhat, y)

        def apply_offset(data, bin_offset, sv, scorer=eval_wrapper):
            # data has the format of pred=0, offset_pred=1, labels=2 in the first dim
            data[1, data[0].astype(int)==sv] = data[0, data[0].astype(int)==sv] + bin_offset
            score = scorer(data[1], data[2])
            return score

In [10]:
from ml_metrics import quadratic_weighted_kappa
